In [21]:
import pandas as pd
from _conf import BASELINE_ARCHIVES, INSTANCE_ARCHIVE
from _utils import get_instance, parse_sample, parse_solution_overview

baseline_data = pd.concat(
    [parse_solution_overview(p) for p in BASELINE_ARCHIVES], ignore_index=True
)
# Set non available sample sizes to None instead of 0 (which would harm the mean values)
baseline_data.loc[baseline_data["Path"].isna(), "SampleSize"] = None
baseline_data

Found the data folders: {'QuBRA/FeatJAR/evaluation-sampling-algorithms/results/2023-08-18_16-43-25/'}
Found the data folders: {'2023-03-01_13-51-03/'}


,ModelID,AlgorithmID,SystemIteration,AlgorithmIteration,InTime,Success,Time,SampleSize,Instance,#Variables,#Clauses,Algorithm,Settings,Path,ArchivePath
0,1,0,1,1,True,True,1620,9.0,calculate,9,15,ACTS-IPOG-FT,t2,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
1,2,0,1,1,True,True,485,9.0,lcm,9,16,ACTS-IPOG-FT,t2,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
2,3,0,1,1,True,True,514,8.0,email,10,17,ACTS-IPOG-FT,t2,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
3,4,0,1,1,True,True,538,10.0,ChatClient,14,20,ACTS-IPOG-FT,t2,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
4,5,0,1,1,True,True,485,13.0,toybox_2006-10-31_23-30-06,16,13,ACTS-IPOG-FT,t2,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945,50,10,1,1,False,True,900008,NaN,freetz,31012,102705,YASA,t2_m10_null,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
2946,50,10,1,2,False,True,900008,NaN,freetz,31012,102705,YASA,t2_m10_null,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
2947,50,10,1,3,False,True,900010,NaN,freetz,31012,102705,YASA,t2_m10_null,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
2948,50,10,1,4,False,True,900009,NaN,freetz,31012,102705,YASA,t2_m10_null,None,./EXTERNAL_INPUT/900_seconds_5_it.zip


In [45]:
from samplns.verify import have_equal_coverage

for instance_name in baseline_data["Instance"].unique().tolist():
    print("Checking instance", instance_name)
    instance = get_instance(instance_name, INSTANCE_ARCHIVE)
    samples = baseline_data[baseline_data["Instance"] == instance_name]
    samples = samples.dropna(subset=["Path"])  # drop all timed out samples
    if samples.empty or len(samples) == 1:
        print("Not enough samples to compare")
        continue
    samples = samples[~samples["Algorithm"].str.startswith("YASA")]
    samples.sort_values(by="Algorithm", inplace=True, ascending=False)
    baseline_sample = parse_sample(
        samples.iloc[0]["ArchivePath"], samples.iloc[0]["Path"]
    )
    for idx in samples.index:
        if "ACTS" in samples.loc[idx]["Algorithm"]:
            continue
        if samples.loc[idx]["Algorithm"].startswith("YASA"):
            continue
        print(
            "Checking sample by",
            samples.loc[idx]["Algorithm"],
            samples.loc[idx]["Settings"],
        )
        sample = parse_sample(samples.loc[idx]["ArchivePath"], samples.loc[idx]["Path"])
        assert len(sample) == samples.loc[idx]["SampleSize"]
        assert all(instance.is_feasible(conf, verbose=True) for conf in sample)
        if not have_equal_coverage(instance, baseline_sample, sample):
            print(samples.loc[idx])
            print(samples.iloc[0])
            raise AssertionError()

Checking instance calculate
Parsing FeatureIDE
Checking sample by Incling t2
Checking sample by Incling t2
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by Incling t2
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by Incling t2
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by Incling t2
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by FIDE-YASA t2_m1_null
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by FIDE-YASA t2_m1_null
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by FIDE-YASA t2_m3_null
Number of tuples covered by a: 38
Number of tuples co

Removing ALT-Feature Type with only one element.
Removing ALT-Feature VncViewer with only one element.


Checking sample byNumber of tuples covered by a: 328 FIDE-Chvatal t2

Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking instance TightVNC
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Parsing FeatureIDE
Checking sample by Incling t2
Checking sample by Incling t2
Number of tuples covered by a: 788
Number of tuples covered by b: 788
Number of tuples covered by both: 788
Checking sample by Incling t2
Number of tuples covered by a: 788
Number of tuples covered by b: 788
Number

Removing ALT-Feature HiddenWgt with only one element.
Removing ALT-Feature GPL with only one element.


Number of tuples covered by a: 823
Number of tuples covered by b: 823
Number of tuples covered by both: 823
Checking sample by FIDE-ICPL t2
Number of tuples covered by a: 823
Number of tuples covered by b: 823
Number of tuples covered by both: 823
Checking sample by FIDE-ICPL t2
Number of tuples covered by a: 823
Number of tuples covered by b: 823
Number of tuples covered by both: 823
Checking sample by FIDE-ICPL t2
Number of tuples covered by a: 823
Number of tuples covered by b: 823
Number of tuples covered by both: 823
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 823
Number of tuples covered by b: 823
Number of tuples covered by both: 823
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 823
Number of tuples covered by b: 823
Number of tuples covered by both: 823
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 823
Number of tuples covered by b: 823
Number of tuples covered by both: 823
Checking sample by FIDE-Chvatal t2
Number o

...

Checking instance
Number of tuples covered by b: 1610352
Number of tuples covered by both: 1610352
 eCos-3-0_i386pc
Parsing FeatureIDE
Checking sample by Incling t2
Number of tuples covered by a: 2910229
Checking sample by Incling t2
Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
Number of tuples covered by a: 2910229Checking sample by Incling t2

Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
Number of tuples covered by a: 2910229Checking sample by Incling t2

Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
Number of tuples covered by a: 2910229Checking sample by Incling t2

Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
Number of tuples covered by a: 2910229
Checking sample by FIDE-YASA t2_m5_null
Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
Number of tuples covered by a: 2910229
Number of tuples covered by b: Check

Disjunction with only one literal removed.
Disjunction with only one literal removed.
Disjunction with only one literal removed.


Checking sample by Incling t2
Number of tuples covered by a: 3765597Checking sample by Incling t2

Number of tuples covered by b: 3765597
Number of tuples covered by both: 3765597
Number of tuples covered by a: 3765597Checking sample by Incling t2

Number of tuples covered by b: 3765597
Number of tuples covered by both: 3765597
Number of tuples covered by a: 3765597
Number of tuples covered by b: Checking sample by Incling t2
3765597
Number of tuples covered by both: 3765597
Number of tuples covered by a: 3765597Checking sample by Incling t2

Number of tuples covered by b: 3765597
Number of tuples covered by both: 3765597
Number of tuples covered by a: 3765597Checking sample by FIDE-YASA t2_m5_null

Number of tuples covered by b: 3765597
Number of tuples covered by both: 3765597
Number of tuples covered by a: 3765597Checking sample by FIDE-YASA t2_m5_null

Number of tuples covered by b: 3765597
Number of tuples covered by both: 3765597
Number of tuples covered by a: 3765597Checking sam

Removing ALT-Feature N_100300__F_100325_xor with only one element.
Removing ALT-Feature N_100300__F_100330 with only one element.
Removing ALT-Feature N_100000__I_101285_i_F_101295 with only one element.
Removing ALT-Feature N_100000__I_101285_i_F_101310 with only one element.
Removing ALT-Feature N_100000__I_101405_i_F_101415 with only one element.
Removing ALT-Feature N_100000__I_101405_i_F_101430 with only one element.
Removing ALT-Feature N_100000__I_101525_i_F_101535 with only one element.
Removing ALT-Feature N_100000__I_101525_i_F_101550 with only one element.
Removing ALT-Feature N_100000__I_101645_i_F_101655 with only one element.
Removing ALT-Feature N_100000__I_101645_i_F_101670 with only one element.
Removing ALT-Feature N_101764__F_101784_xor with only one element.
Removing ALT-Feature N_101764__F_101808 with only one element.
Removing ALT-Feature N_101906__F_101927 with only one element.
Removing ALT-Feature N_101932__F_101952 with only one element.
Removing ALT-Feature N

Checking instance Automotive01
2016082
Parsing FeatureIDE
Checking sample by FIDE-YASA t2_m1_null
Number of tuples covered by a: 5872250Checking sample by FIDE-YASA t2_m1_null

Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
Number of tuples covered by a: 5872250
Checking sample by FIDE-YASA t2_m10_null
Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
Number of tuples covered by a: 5872250
Checking sample by FIDE-YASA t2_m10_null
Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
Checking sample byNumber of tuples covered by a: 5872250
Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
 FIDE-YASA t2_m10_null
Number of tuples covered by a: 5872250
Number of tuples covered by b: Checking sample by FIDE-YASA t2_m10_null
5872250
Number of tuples covered by both: 5872250
Number of tuples covered by a: 5872250Checking sample by FIDE-YASA t2_m5_null

Number of tuples co

In [46]:
bad_instances = []

for instance_name in baseline_data["Instance"].unique().tolist():
    print("Checking instance", instance_name)
    instance = get_instance(instance_name, INSTANCE_ARCHIVE)
    samples = baseline_data[baseline_data["Instance"] == instance_name]
    samples = samples.dropna(subset=["Path"])  # drop all timed out samples
    if samples.empty or len(samples) == 1:
        print("Not enough samples to compare")
        continue
    # samples = samples[~samples["Algorithm"].str.startswith("YASA")]
    samples.sort_values(by="Algorithm", inplace=True, ascending=False)
    baseline_sample = parse_sample(
        samples.iloc[0]["ArchivePath"], samples.iloc[0]["Path"]
    )
    for idx in samples.index:
        if "ACTS" in samples.loc[idx]["Algorithm"]:
            continue
        print(
            "Checking sample by",
            samples.loc[idx]["Algorithm"],
            samples.loc[idx]["Settings"],
        )
        sample = parse_sample(samples.loc[idx]["ArchivePath"], samples.loc[idx]["Path"])
        assert len(sample) == samples.loc[idx]["SampleSize"]
        assert all(instance.is_feasible(conf, verbose=True) for conf in sample)
        if not have_equal_coverage(instance, baseline_sample, sample):
            print(samples.loc[idx])
            print(samples.iloc[0])
            bad_instances.append(instance_name)
            break

Checking instance calculate
Parsing FeatureIDE
Checking sample by YASA t2_m10_null
Checking sample by YASA t2_m3_null
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by YASA t2_m1_null
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by YASA t2_m1_null
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by YASA t2_m1_null
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by YASA t2_m1_null
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by YASA t2_m1_null
Number of tuples covered by a: 38
Number of tuples covered by b: 38
Number of tuples covered by both: 38
Checking sample by YASA t2_m3_null
Number of tuples covered by a: 38
Number o

Checking sample byNumber of tuples covered by a: 37
Number of tuples covered by b: 37
Number of tuples covered by both: 37
 YASA t2_m10_null
Checking sample by YASA t2_m10_null
Number of tuples covered by a: 37
Number of tuples covered by b: 37
Number of tuples covered by both: 37
Checking sample by YASA t2_m3_null
Number of tuples covered by a: 37
Number of tuples covered by b: 37
Number of tuples covered by both: 37
Checking sample by Incling t2
Number of tuples covered by a: 37
Number of tuples covered by b: 37
Number of tuples covered by both: 37
Checking sample by Incling t2
Number of tuples covered by a: 37
Number of tuples covered by b: 37
Number of tuples covered by both: 37
Checking sample by Incling t2
Number of tuples covered by a: 37
Number of tuples covered by b: 37
Number of tuples covered by both: 37
Checking sample by Incling t2
Number of tuples covered by a: 37
Number of tuples covered by b: 37
Number of tuples covered by both: 37
Checking sample by Incling t2
Number o

Removing ALT-Feature Type with only one element.
Removing ALT-Feature VncViewer with only one element.


Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking sample by FIDE-Chvatal t2
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Checking instance TightVNC
Number of tuples covered by a: 328
Number of tuples covered by b: 328
Number of tuples covered by both: 328
Parsing FeatureIDE
Checking sample by YASA t2_m10_null
Checking sample by YASA t2_m3_null
Number of tuples covered by a: 788
Number of tuples covered by b: 788
Number of tuples covered 

Removing ALT-Feature HiddenWgt with only one element.
Removing ALT-Feature GPL with only one element.


...

Number of tuples covered by a: 1610352
Checking instance eCos-3-0_i386pc
Number of tuples covered by b: 1610352
Number of tuples covered by both: 1610352
Parsing FeatureIDE
Checking sample by YASA t2_m10_null
Number of tuples covered by a: 2910229
Checking sample by YASA t2_m5_null
Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
Number of tuples covered by a: 2910229Checking sample by
Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
 YASA t2_m1_null
Number of tuples covered by a: 2910229Checking sample by YASA t2_m1_null

Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
Number of tuples covered by a: 2910229Checking sample by YASA t2_m1_null

Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
Number of tuples covered by a: 2910229Checking sample by
Number of tuples covered by b: 2910229
Number of tuples covered by both: 2910229
 YASA t2_m1_null
Number of tuples

Disjunction with only one literal removed.
Disjunction with only one literal removed.
Disjunction with only one literal removed.


Checking sample by YASA t2_m10_null
Number of tuples covered by a: 3765453
Number of tuples covered by b: 3765453
Checking sample by YASA t2_m5_null
Number of tuples covered by both: 3765453
Number of tuples covered by a: 6472
Number of tuples covered by b: ModelID                                                    46
AlgorithmID                                                 9
SystemIteration                                             1
AlgorithmIteration                                          2
InTime                                                   True
Success                                                  True
Time                                                    44873
SampleSize                                               76.0
Instance                                        FreeBSD-8_0_0
#Variables                                               1397
#Clauses                                                15692
Algorithm                                                YAS

Removing ALT-Feature N_100300__F_100325_xor with only one element.
Removing ALT-Feature N_100300__F_100330 with only one element.
Removing ALT-Feature N_100000__I_101285_i_F_101295 with only one element.
Removing ALT-Feature N_100000__I_101285_i_F_101310 with only one element.
Removing ALT-Feature N_100000__I_101405_i_F_101415 with only one element.
Removing ALT-Feature N_100000__I_101405_i_F_101430 with only one element.
Removing ALT-Feature N_100000__I_101525_i_F_101535 with only one element.
Removing ALT-Feature N_100000__I_101525_i_F_101550 with only one element.
Removing ALT-Feature N_100000__I_101645_i_F_101655 with only one element.
Removing ALT-Feature N_100000__I_101645_i_F_101670 with only one element.
Removing ALT-Feature N_101764__F_101784_xor with only one element.
Removing ALT-Feature N_101764__F_101808 with only one element.
Removing ALT-Feature N_101906__F_101927 with only one element.
Removing ALT-Feature N_101932__F_101952 with only one element.
Removing ALT-Feature N

Checking instance Automotive01

Number of tuples covered by b: 2016082
Number of tuples covered by both: 2016082
Parsing FeatureIDE
Checking sample by YASA t2_m1_null
Number of tuples covered by a: 5872250Checking sample by YASA t2_m5_null

Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
Number of tuples covered by a: 5872250Checking sample by YASA t2_m1_null

Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
Number of tuples covered by a: 5872250Checking sample by YASA t2_m1_null

Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
Number of tuples covered by a: 5872250Checking sample by YASA t2_m3_null

Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
Checking sample byNumber of tuples covered by a: 5872250
Number of tuples covered by b: 5872250
Number of tuples covered by both: 5872250
 YASA t2_m3_null
Number of tuples covered by a: 5872250Checking sample by Y

In [43]:
bad_instances

['uclibc_2008-06-05_13-46-47',
 'uclibc_2020-12-24_11-54-53',
 'financial_services',
 'EMBToolkit',
 'FreeBSD-8_0_0',
 'Automotive01',
 'main_light']